### Import necessary libraries and load dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load datasets
users = pd.read_csv('Users.csv')
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')

C:\Users\suman\AppData\Local\Temp\ipykernel_18352\651844884.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [4]:
# Get dataset info
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


### Data Cleaning and Preparation

In [7]:
new_books = books.drop_duplicates('Book-Title')

In [8]:
# Merge ratings and new_books df
ratings_with_name = ratings.merge(new_books, on='ISBN')

In [9]:
ratings_with_name.head()

User-ID        ISBN  Book-Rating            Book-Title Book-Author  \
0   276725  034545104X            0  Flesh Tones: A Novel  M. J. Rose   
1     2313  034545104X            5  Flesh Tones: A Novel  M. J. Rose   
2     6543  034545104X            0  Flesh Tones: A Novel  M. J. Rose   
3     8680  034545104X            5  Flesh Tones: A Novel  M. J. Rose   
4    10314  034545104X            9  Flesh Tones: A Novel  M. J. Rose   

  Year-Of-Publication         Publisher  \
0                2002  Ballantine Books   
1                2002  Ballantine Books   
2                2002  Ballantine Books   
3                2002  Ballantine Books   
4                2002  Ballantine Books   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/034545104X.0...   
1  http://images.amazon.com/images/P/034545104X.0...   
2  http://images.amazon.com/images/P/034545104X.0...   
3  http://images.amazon.com/images/P/034545104X.0...   
4  http://images.amazon.com/images/P/034545104X.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/034545104X.0...   
1  http://images.amazon.com/images/P/034545104X.0...   
2  http://images.amazon.com/images/P/034545104X.0...   
3  http://images.amazon.com/images/P/034545104X.0...   
4  http://images.amazon.com/images/P/034545104X.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/034545104X.0...  
1  http://images.amazon.com/images/P/034545104X.0...  
2  http://images.amazon.com/images/P/034545104X.0...  
3  http://images.amazon.com/images/P/034545104X.0...  
4  http://images.amazon.com/images/P/034545104X.0...

In [10]:
# Drop non-relevant columns
ratings_with_name.drop(['ISBN', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis = 1, inplace = True)

In [12]:
ratings_with_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883079 entries, 0 to 883078
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   User-ID              883079 non-null  int64 
 1   Book-Rating          883079 non-null  int64 
 2   Book-Title           883079 non-null  object
 3   Book-Author          883078 non-null  object
 4   Year-Of-Publication  883079 non-null  object
 5   Publisher            883078 non-null  object
dtypes: int64(2), object(4)
memory usage: 47.2+ MB


In [13]:
# Merge new 'ratings_with_name' df with users df
users_ratings_matrix = ratings_with_name.merge(users, on='User-ID')

In [14]:
# Drop non-relevant columns
users_ratings_matrix.drop(['Location', 'Age'], axis = 1, inplace = True)

In [15]:
# Print the first few rows of the new dataframe
users_ratings_matrix.head()

User-ID  Book-Rating                                        Book-Title  \
0   276725            0                              Flesh Tones: A Novel   
1     2313            5                              Flesh Tones: A Novel   
2     2313            8             In Cold Blood (Vintage International)   
3     2313            9  Divine Secrets of the Ya-Ya Sisterhood : A Novel   
4     2313            5                            The Mistress of Spices   

                  Book-Author Year-Of-Publication               Publisher  
0                  M. J. Rose                2002        Ballantine Books  
1                  M. J. Rose                2002        Ballantine Books  
2               TRUMAN CAPOTE                1994                 Vintage  
3               Rebecca Wells                1996           HarperCollins  
4  Chitra Banerjee Divakaruni                1998  Anchor Books/Doubleday

In [16]:
users_ratings_matrix.isna().sum()

User-ID                0
Book-Rating            0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              1
dtype: int64

In [17]:
users_ratings_matrix.dropna(inplace = True)

In [18]:
# Filter down 'users_ratings_matrix' on the basis of users who gave many book ratings
x = users_ratings_matrix.groupby('User-ID').count()['Book-Rating'] > 100
knowledgeable_users = x[x].index
filtered_users_ratings = users_ratings_matrix[users_ratings_matrix['User-ID'].isin(knowledgeable_users)]

In [19]:
# Filter down 'users_ratings_matrix' on the basis of books with most ratings
y = filtered_users_ratings.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index
final_users_ratings = filtered_users_ratings[filtered_users_ratings['Book-Title'].isin(famous_books)]

In [20]:
# Pivot table creation
pivot_table = final_users_ratings.pivot_table(index = 'Book-Title', columns = 'User-ID', values = 'Book-Rating')

In [21]:
# Filling the NA values with '0'
pivot_table.fillna(0, inplace = True)
pivot_table.head()

User-ID              254     507     882     1424    1435    1733    1903    \
Book-Title                                                                    
1984                    9.0     0.0     0.0     0.0     0.0     0.0     0.0   
1st to Die: A Novel     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2010: Odyssey Two       0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2nd Chance              0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4 Blondes               0.0     0.0     0.0     0.0     0.0     0.0     0.0   

User-ID              2033    2110    2276    ...  274808  275020  275970  \
Book-Title                                   ...                           
1984                    0.0     0.0     0.0  ...     0.0     0.0     0.0   
1st to Die: A Novel     0.0     0.0     0.0  ...     0.0     0.0     0.0   
2010: Odyssey Two       0.0     0.0     0.0  ...     0.0     0.0     0.0   
2nd Chance              0.0     0.0     0.0  ...     0.0     0.0     0.0   
4 Blondes               0.0     0.0     0.0  ...     0.0     0.0     0.0   

User-ID              276680  277427  277478  277639  278137  278188  278418  
Book-Title                                                                   
1984                    0.0     0.0     0.0     0.0     0.0     0.0     0.0  
1st to Die: A Novel     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
2010: Odyssey Two       0.0     0.0     0.0     0.0     0.0     0.0     0.0  
2nd Chance              0.0     0.0     0.0     0.0     0.0     0.0     0.0  
4 Blondes               0.0     0.0     0.0     0.0     0.0     0.0     0.0  

[5 rows x 1391 columns]

In [22]:
# Standardize the pivot table
scaler = StandardScaler(with_mean=True, with_std=True)
pivot_table_normalized = scaler.fit_transform(pivot_table)